## For some variables, it would be more useful to know the relationship between them at *daily* scales rather than using monthly averages. 
<b>Author:</b> Meg Fowler <br>
<b>Date:</b> 26 Oct 2020 <br><br>

In [1]:
# Load libraries

# Plotting utils 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import matplotlib.ticker as ticker 
import matplotlib.patches as patches
import matplotlib.animation as animation
import matplotlib as matplotlib
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.util

# Analysis
import os
import numpy as np 
import xarray as xr
import datetime
from   datetime import date, timedelta
import pandas as pd 
#import regionmask
import pickle
import scipy.stats as stats


## 1. Read in data 

Note that for the UV daily data, since this is such a large dataset, only the lowest three levels were selected via the NCO command: <br> 
<i>ncrcat -d lev,950.0,999.0 -v U,V f.e20.FHIST.f09_f09.cesm2_1.001.cam.h1.199* f.e20.FHIST.f09_f09.cesm2_1.001.cam.h1.1990-99_sfcLevs-dailyUV.nc</i> <br><br>
Once those files have been read in here, a new pickle file is created that contains the total surface wind speed, rather than U and V seperately. Once that pickle file is available, the original UV files will be deleted. 

In [8]:
# Set up directories 
dataDir    = '/Users/mdfowler/Documents/Analysis/CLUBB_initial/data/daily/'
nameStart  = 'f.e20.FHIST.f09_f09.cesm2_1.001.cam.h1.'
nameEnd_UV = '_sfcLevs-dailyUV.nc'
nameEnd_FLX  = '_dailySfcFluxes.nc'

#decadeList = ['1970-79','1980-89','1990-99']
decadeList = ['1970-79']

In [4]:
# Read in single history file to get lat/lon and masks 
testName = '/Users/mdfowler/Documents/Analysis/CLUBB_initial/data/f.e20.FHIST.f09_f09.cesm2_1.001.clm2.h0.1989-12.nc'
testDF   = xr.open_dataset(testName)

# Set lat, lon 
lat = testDF.lat
lon = testDF.lon

# Make land mask
landMask              = testDF.landmask.values
landMask[landMask==0] = np.nan

# Ideally, want to mask out Greenland/Antarctica too (ice sheets)
iceContent = testDF.ICE_CONTENT1.values[0,:,:]  # "Initial gridcell total ice content"
iceMask    = np.full([len(lat),len(lon)], np.nan)
iceMask[iceContent<10000] = 1

<ipython-input-4-a62247b44c48>:16: RuntimeWarning: invalid value encountered in less
  iceMask[iceContent<10000] = 1


In [7]:
# Read in data by decade 
for iDec in range(len(decadeList)): 
    
    # Open datasets for each decade 
    UVfile = dataDir+nameStart+decadeList[iDec]+nameEnd_UV 
    windDF = xr.open_dataset(UVfile, decode_times=True)
    windDF['time'] = windDF.indexes['time'].to_datetimeindex()   
    
    FLXfile = dataDir+nameStart+decadeList[iDec]+nameEnd_FLX
    flxDF   = xr.open_dataset(FLXfile, decode_times=True)
    flxDF['time'] = flxDF.indexes['time'].to_datetimeindex()
    
    # Create *giant* datasets that span the full period of the simulations
    if iDec==0:
        fullUV   = windDF
        fullFLX  = flxDF
    else:
        fullUV   = xr.concat([fullUV, windDF], dim="time")
        fullFLX  = xr.concat([fullFLX, flxDF], dim="time")
        
    # Close files 
    windDF.close() 
    flxDF.close()

        
    print('Done with decade ', decadeList[iDec], ' ... ')

<ipython-input-7-6e24db6f250f>:7: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  windDF['time'] = windDF.indexes['time'].to_datetimeindex()
<ipython-input-7-6e24db6f250f>:11: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  flxDF['time'] = flxDF.indexes['time'].to_datetimeindex()


Done with decade  1970-79  ... 
Done with decade  1980-89  ... 


KeyboardInterrupt: 

In [ ]:
# To be able to easily access years, months, days - use Pandas 
dates = pd.DatetimeIndex(fullDF['time'].values) 

# Let's set the monthly averages to be roughly mid-month
#   This way, the average for January has a month of 1 instead of being the first day of February 
midTime = dates - timedelta(days=15)       # Get new dates array that has the right month/year in it 



In [ ]:
# Read into individual arrays for easy access
SHFLX = fullFLX.SHFLX.values
LHFLX = fullFLX.LHFLX.values
U     = fullUV.U.values              # Zonal wind (m/s)
V     = fullUV.V.values              # Meridional wind (m/s)

# Flip along vertical (level) axis, so that index 0 is surface 
U   = np.flip(U, axis=1)
V   = np.flip(V, axis=1)

# Save levels themselves to arrays and flip them 
lev_middle    = np.flip(fullUV.lev.values)


In [ ]:
# Now get surface wind magnitude and combine variances 
windSpd = np.sqrt(U**2 + V**2)


In [ ]:
# Save wind speed out to a pickle file
fileOutName = dataDir+'totalWindSpeeds.p'
pickle.dump( [windSpd,lev_middle], open( fileOutName, "wb" ), protocol=4 )


In [ ]:
# Load wind speed from pickle file: 
fileOutName = dataDir+'totalWindSpeeds.p'
windSpd, lev_middle = pickle.load( open (fileOutName, "r"))


### 1.1 Compute and save mean wind speed rather than looking at zonal/meridional components seperately 